In [ ]:
"""
this notebook is to analyze the dino embeddings of imagenet images
"""

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoImageProcessor
import matplotlib.pyplot as plt
import os, sys
import math
sys.path.append(os.path.abspath(".."))  # allow relative imports if needed
print(os.path.abspath(".."))
from collections import defaultdict

from data.utils.dataloaders import get_imagenet_dataloader
import torch.nn.functional as F

/BS/data_mani_compress/work/miniforge3/envs/dgm_geometry/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/BS/data_mani_compress/work/thesis/thesis


In [ ]:
imagenet_original_dataloader = get_imagenet_dataloader(split="val_categorized", batch_size=64, shuffle=False, num_workers=2)

In [6]:
# read .pt file
dino_features_path = "../data/datasets/dino_embeddings/val_categorized/dino_features.pt"
dino_features = torch.load(dino_features_path)  # shape [N, D]
print(f"Loaded DINO features with shape: {dino_features.shape}")  # [N, D]

/tmp/ipykernel_2246566/1851463452.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dino_features = torch.load(dino_features_path)  # shape [N, D]


Loaded DINO features with shape: torch.Size([50000, 768])


In [14]:
# get all the labels from the dataloader and load them into a tensor
all_labels = []
for _, labels in imagenet_original_dataloader:
    all_labels.append(labels)
all_labels = torch.cat(all_labels, dim=0)  # shape [N]
print(f"Loaded labels with shape: {all_labels.shape}")  # [N]

Loaded labels with shape: torch.Size([50000])


In [28]:
normalized_features = F.normalize(dino_features, p=2, dim=1)  # normalize features for cosine similarity
print(f"Normalized features shape: {normalized_features.shape}")  # [N, D]

Normalized features shape: torch.Size([50000, 768])


In [ ]:
# group the features by their labels
features_by_label = defaultdict(list)
for feature, label in zip(normalized_features, all_labels):
    features_by_label[label.item()].append(feature)

In [30]:
# find the mean and variance of the features for each label
mean_dist_by_label = {}
std_dist_by_label = {}
for label, features in features_by_label.items(): # get the key and value
    features_tensor = torch.stack(features, dim=0)  # shape [num_samples, D]
    class_mean = features_tensor.mean(dim=0, keepdim=True)  # [1, D]

    # Euclidean distances to class mean
    dists = (features_tensor - class_mean).pow(2).sum(dim=1).sqrt()  # [n]

    mean_dist_by_label[label] = dists.mean()  # scalar per class
    std_dist_by_label[label] = dists.std()

In [34]:
# print out the index of the class with the highest mean distance
max_mean_dist_label = max(mean_dist_by_label, key=mean_dist_by_label.get)
print(f"Class with highest mean distance to centroid: {max_mean_dist_label}, Mean Distance: {mean_dist_by_label[max_mean_dist_label]:.4f}, Std Dev: {std_dist_by_label[max_mean_dist_label]:.4f}")

min_mean_dist_label = min(mean_dist_by_label, key=mean_dist_by_label.get)
print(f"Class with lowest mean distance to centroid: {min_mean_dist_label}, Mean Distance: {mean_dist_by_label[min_mean_dist_label]:.4f}, Std Dev: {std_dist_by_label[min_mean_dist_label]:.4f}")

Class with highest mean distance to centroid: 309, Mean Distance: 0.9515, Std Dev: 0.0322
Class with lowest mean distance to centroid: 995, Mean Distance: 0.3729, Std Dev: 0.1168
